# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

In [5]:
# view in a dataframe
df = pd.read_csv("event_datafile_new.csv")
df.head(3)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88


In [6]:
# view data types for table creation below
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify
                    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### Create table for query 1:
* sessionID and ItemInSession are considerd composite keys because the combination of the two creates a unique row which can identify the items for sesssion Id and itemInSession.

In [10]:
# check for existing table
dropme = "DROP TABLE IF EXISTS music_hist_session"

try:
    session.execute(dropme)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS music_hist_session"

query = query + "(sessionId int, itemInSession int, artist text, song text, length double, PRIMARY KEY(sessionId, itemInSession))" 

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Insert data into table for query/question 1:

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_hist_session(sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES(%s, %s,%s, %s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Question 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [12]:
query = "SELECT artist, song, length FROM music_hist_session WHERE sessionId = 338 AND itemInSession = 4 "

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Create table for query/question 2:
* The Primary Key used is the user id given the query search is based on the user but  cluster of colunns is also created to use the sesssion id as a unique identifer with the user id and the itemInSession cluster column will sort the sessions.

In [13]:
# check for existing table
dropme = "DROP TABLE IF EXISTS song_playlist_session"

try:
    session.execute(dropme)
except Exception as e:
    print(e)

query2 = "CREATE TABLE IF NOT EXISTS song_playlist_session"

query2 = query2 + "(userId int, itemInSession int, sessionId int, artist text, song text, firstName text, \
                    lastName text, PRIMARY KEY((userId), sessionId, itemInSession))" 

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)    

### Insert data into table for query 2:

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session(userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES(%s, %s,%s, %s,%s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
query_2 = "SELECT artist, song, firstName, lastName FROM song_playlist_session WHERE userid = 10 AND sessionid = 182 "

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Create table for query/question 3:
* Song and userid create a composite key which allow a row return for each unique user that listened to the song that used in the where statment.

In [21]:
# check for existing table
dropme = "DROP TABLE IF EXISTS users_song_all_hands_q3"

try:
    session.execute(dropme)
except Exception as e:
    print(e)

query3 = "CREATE TABLE IF NOT EXISTS users_song_all_hands_q3"
query3 = query3 + "(song text, userId int, firstName text, \
                    lastName text, PRIMARY KEY(song, userId))" 

try:
    rows = session.execute(query3)
except Exception as e:
    print(e) 
           

### Insert the data into table for query 3:

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_song_all_hands_q3(song, userId, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s,%s)"
        
        session.execute(query, (line[9], int(line[10]),line[1],line[4]))

### Question 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [25]:
query_3 = "SELECT firstName, lastName FROM users_song_all_hands_q3 WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [26]:
drop_query1 = "DROP TABLE music_hist_session"

try:
    session.execute(drop_query1)
except Exception as e:
    print(e)
    
drop_query2 = "DROP TABLE song_playlist_session"

try:
    session.execute(drop_query2)
except Exception as e:
    print(e)
    
drop_query3 = "DROP TABLE users_song_all_hands_q3"

try:
    session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()